In [1]:
import scrapy
import requests
from scrapy.http import TextResponse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from scrapy.selector import Selector
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from bs4 import BeautifulSoup
import urllib.request
import warnings
warnings.filterwarnings("ignore")

In [2]:
# from fake_useragent import UserAgent
# url = "https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from=2022&release_year_until=2022"
# headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" }
# req = requests.get(url, headers=headers)
# response = TextResponse(req.url, body=req.text, encoding="utf-8") 
# response

In [2]:
def get_chrome_driver():
    # 1. 크롬 옵션 세팅
    chrome_options = webdriver.ChromeOptions()
    WINDOW_SIZE = "1920,1080"
    
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument(f"--window-size={WINDOW_SIZE}")
    # 2. driver 생성하기
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), # 가장 많이 바뀐 부분
        options=chrome_options
    )
    
    return driver

In [3]:
driver = get_chrome_driver()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Kims\.wdm\drivers\chromedriver\win32\99.0.4844.51]


In [4]:
urls = []
for year in map(str,range(2005,2006)):
    url = 'https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from={}&release_year_until={}'.format(year, year)
    urls.append(url)

In [5]:
for url in urls:
    driver.get(url)
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # 1초 대기
        time.sleep(1)
        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height        

        elem = driver.find_elements_by_css_selector('div.title-list-grid div.title-list-grid__item a.title-list-grid__item--link')
        linkUrls = []
        for e in elem:
            linkUrl = e.get_attribute('href')
            linkUrls.append(linkUrl)    

In [6]:
len(linkUrls)

44

In [7]:
title_list = []
openingdate_list = []
just_rating_list = []
imdb_rating_list = []
genre_list = []
runtime_list = []
director_list = []
synopsis_list = []
actors_list = []
poster_list = []

In [8]:
for i in range(0,len(linkUrls)):
    url = linkUrls[i]
    headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" }
    req = requests.get(url, headers=headers)
    response = TextResponse(req.url, body=req.text, encoding="utf-8")
    response

    
    title_kor = response.css("div.title-block > div > h1::text").get().strip()        
    opening_date = response.css("div.title-block > div > span::text").get().replace("(","").replace(")","").strip()

    try:
        just_rating = response.css('div.detail-infos > div > div > div > a::text')[0].get().strip()
    except:
        just_rating = None
    try:
        imdb_rating = response.css('div.detail-infos > div > div > div > a::text')[1].get().strip()
    except:
        imdb_rating = None   
    genre = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div/div[2]/span/text()').getall()     # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
    genre = ",".join(genre).replace(" ","")
    genre = genre
    runtime = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[2]/text()').getall()
    runtime = ",".join(runtime).replace(" ","")
    runtime = runtime
    director = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[4]/div[2]/span/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
    director = ",".join(director).replace(" ","")
    director = director
    actors = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[3]/div/div/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음...
    actors = ",".join(actors).replace(" ","")
    actors = actors
    
    try:
        synopsis = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[4]/p/span/text()').get()
    except:
        synopsis= None

    # poster = response.xpath('//*[@id="base"]/div[2]/div/div[1]/div/aside/div[2]/div/picture/img/@data-src').get()
    poster = response.css('source').getall()
    
    
    title_list.append(title_kor)
    openingdate_list.append(opening_date)
    just_rating_list.append(just_rating)
    just_rating_list.append(imdb_rating)
    genre_list.append(genre)
    runtime_list.append(runtime)
    director_list.append(director)
    synopsis_list.append(synopsis)
    actors_list.append(actors)
    poster_list.append(poster)

In [9]:
title_list

['Kureyon Shinchan: Densetsu o Yobu Buriburi: Sanpun Bokkiri Daishingeki',
 '콘스탄틴',
 '오만과 편견',
 '찰리와 초콜릿 공장',
 '브로크백 마운틴',
 '마다가스카',
 '디센트',
 '우주전쟁',
 '연애의 목적',
 '유령 신부',
 '자헤드: 그들만의 전쟁',
 '미스터 & 미세스 스미스',
 '거북이는 의외로 빨리 헤엄친다',
 '친절한 금자씨',
 '게이샤의 추억',
 '시노비',
 '저스트 프렌드',
 '말아톤',
 '오징어와 고래',
 '아미티빌 호러',
 '혈의 누',
 '내 생애 가장 아름다운 일주일',
 '웰컴 투 동막골',
 '용서받지 못한 자',
 '주먹이 운다',
 '이니셜 D',
 '나나',
 '태극권',
 '흑사회',
 '마파도',
 '미스터 소크라테스',
 '왕의 남자',
 '메종 드 히미코',
 '분홍신',
 '극장전',
 '가문의 영광 2 - 가문의 위기',
 '야수와 미녀',
 '작업의 정석',
 '박수칠 때 떠나라',
 '그때 그 사람들',
 '태풍',
 '루팡 3세 천사의 책략 - 꿈의 조각은 살인의 향기',
 '미스터 주부 퀴즈왕',
 '공공의 적 2']

In [10]:
poster_list

In [61]:
print("제목:",title_kor, "개봉일:",opening_date,"저스트와치 평점:", just_rating,"IMDB 평점:",imdb_rating, "장르:",genre, "재생시간:",runtime, "감독:",director,"출연자:", actors, "시놉시스:",synopsis)
type(title_kor)

제목: 공공의 적 2 개봉일: 2005 저스트와치 평점: 72% IMDB 평점: 6.0 장르: ['스릴러 ', '드라마 ', '액션 '] 재생시간: 2시간 28분 감독: [' Kang Woo-suk '] 출연자: [' Sol Kyung-gu ', ' Jung Joon-ho ', ' Kang Shin-il ', ' Park Geun-hyung ', ' Byun Hee-bong ', ' Park Sang-wook ', ' Uhm Tae-woong ', ' Jeong Gyu-su ', ' Choi Jung-woo ', ' Park Woong '] 시놉시스: 검찰청 최고의 꼴통검사 강철중, 룸싸롱은 주로 쳐들어 가기 위해 자주 들르고 책상머리의 서류철 들여다보기 보다 컵라면 우물거리며 잠복근무가 체질인 그는 나쁜 짓 하는 놈 잡기 위해 부하와 동료의 피를 볼 순 없다는 일념으로 총기류 사용도 마다하지 않는 다혈질 검사다. 그렇지만 어제도 오늘도 그의 하루는 늘 나름대로 스마일~로 시작된다.  그런 그에게 명선 재단 이사장 한상우 사건이 접수되고 특유의 기질로 나쁜 냄새를 직감한 강검사, 자기 담당도 아닌 사건을 조사하기 시작한다. 명선고교는 다름아닌 강검사의 모교이며 학교 재단 이사장의 둘째 아들인 한상우 또한 자신과 3년 동안 같은 반에서 수학한 동기동창이었던 것. 재단을 물려받기로 되어 있던 명선 재단 큰 아들의 사고로 인해 재단 이사장으로 급부상한 한상우, 때마침 명선 재단 큰 아들이 숨을 거두고 그 죽음에 대한 의문이 제기되면서 강철중 검사의 수사는 큰 파문을 불러 일으킨다. 이에 철중은 조사인의 자격으로 상우를 검찰청에서 마주하게 되는데.


str

In [ ]:
import pandas as pd
# df = pd.DataFrame({
#     "제목":title_list,"개봉일":openingdate_list,"저스트와치 평점":just_rating_list,"장르":genre_list,"재생시간":runtime_list, "감독":director_list,"시놉시스":synopsis_list})
# df.tail(5)
df = pd.DataFrame({
    "제목":title_list,"개봉일":openingdate_list})
df

In [1]:
driver.quit()


NameError: name 'driver' is not defined

In [ ]:
# csv파일 읽어오기
import pandas as pd
# csv_test = pd.read_csv("C:/Users/Kims/Desktop/HK_Project_Team5/movies.csv")
csv_test = pd.read_csv(r'C:\Workspace\AtHome\movieScrapy\movieScrapy\spiders\assets\movies.csv')


In [ ]:
csv_test.tail(20)

In [ ]:
urls = []
for year in map(str,range(1962,2023)):
    ss = 'https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from={}&release_year_until={}'.format(year, year)
    urls.append(ss)

urls

In [1]:
# csv파일 읽어오기 ( 웨이브 )
import pandas as pd
# csv_test = pd.read_csv("C:/Users/Kims/Desktop/HK_Project_Team5/movies.csv")
csv_test_2 = pd.read_csv(r"C:\Users\Kims\Desktop\HK_Project_Team5\wavve_test2차.csv")
len(csv_test_2)

10792

In [3]:
csv_test_2.tail(3)

,title_kor,opening_date,just_rating,imdb_rating,runtime,synopsis,director,actors,genre,posterLink
10789,특명:스피드레스큐,2021.0,NaN,NaN,1시간22분,2021년 상반기를 장악할 초대형 밀리터리 액션 무비! 거대 범죄 조직 14D를 제...,KukKok-Leung,"WangHaixiang,YanJie,YanJiaying,ChenMeilin,Peng...",액션,https://images.justwatch.com/poster/243595035/...
10790,싱어송,2022.0,NaN,NaN,1시간42분,노래가 하고 싶어 무작정 서울로 상경한 ‘선혜’와\n 하루 빨리 유명해지고 싶은 무...,NaO-heon,"HeoGa-yoon,YooYoung-jae,MoonJiHoo",드라마,https://images.justwatch.com/poster/259943680/...
10791,나의 촛불,2022.0,65%,65%,1시간27분,2016년 전세계가 주목한 그 시작의 날 진보와 보수를 넘나드는 정치인들이 그날의...,"JooJin-woo,KimEui-sung","SohnSuk-hee,YooSi-min,ChooMi-ae,SimSang-jung,Y...",다큐멘터리,https://images.justwatch.com/poster/263027725/...


In [9]:
csv_test_2.isnull().sum()

title_kor        59
opening_date     59
just_rating     665
imdb_rating     665
runtime          59
synopsis         86
director        442
actors          539
genre           250
posterLink      124
dtype: int64

In [13]:
# 시놉시스가 없는 영화 확인
csv_test_2['synopsis'].isnull().sum()

86

In [14]:
# 시놉시스 없는 영화 버리기
wavve2 = csv_test_2.dropna(subset=['synopsis'])

In [15]:
# 포스터 링크 없는 영화 버기
wavve2 = csv_test_2.dropna(subset=['posterLink'])

In [16]:
len(wavve2)
# wavve2.isnull().sum()

10668

In [17]:
wavve2.isnull().sum()

title_kor         0
opening_date      0
just_rating     554
imdb_rating     554
runtime           0
synopsis         16
director        346
actors          432
genre           167
posterLink        0
dtype: int64

In [19]:
# 결측치 제거 후 인덱스 리셋 
wavve2.reset_index(drop=True, inplace=True)

In [ ]:
wavve2['netflix'] = 0
wavve2['disney-plus'] = 0
wavve2['wavve'] = 1
wavve2['watcha'] = 0

In [23]:
wavve2.tail(2)

,title_kor,opening_date,just_rating,imdb_rating,runtime,synopsis,director,actors,genre,posterLink,netflix,disney-plus,wavve,watcha
10666,싱어송,2022.0,NaN,NaN,1시간42분,노래가 하고 싶어 무작정 서울로 상경한 ‘선혜’와\n 하루 빨리 유명해지고 싶은 무...,NaO-heon,"HeoGa-yoon,YooYoung-jae,MoonJiHoo",드라마,https://images.justwatch.com/poster/259943680/...,0,0,1,0
10667,나의 촛불,2022.0,65%,65%,1시간27분,2016년 전세계가 주목한 그 시작의 날 진보와 보수를 넘나드는 정치인들이 그날의...,"JooJin-woo,KimEui-sung","SohnSuk-hee,YooSi-min,ChooMi-ae,SimSang-jung,Y...",다큐멘터리,https://images.justwatch.com/poster/263027725/...,0,0,1,0


In [24]:
wavve2.to_csv("wavve.csv")
wavve2.to_excel("wavve.xlsx")

In [49]:
# csv파일 읽어오기 ( 왓챠 )
import pandas as pd
# csv_test = pd.read_csv("C:/Users/Kims/Desktop/HK_Project_Team5/movies.csv")
watcha1 = pd.read_csv(r"C:\Users\Kims\Desktop\HK_Project_Team5\watcha1.csv")
watcha1.tail(3)


,actors,director,genre,imdb_rating,just_rating,opening_date,posterLink,runtime,synopsis,title_kor
5647,"HidekiNagai,ItsukiŌtaki,ToshiakiInomata,TokiKo...",TakeshiKushida,"스릴러,공포,로맨스,코미디",6.3,73%,2022.0,https://images.justwatch.com/poster/226608899/...,1시간29분,"시간이 멈춘듯한 아버지가 남긴 사진관에서, 사진 보정 작업을 하고 있는 여성 공포증...",사진사의 여자
5648,"VanessaAngel,BillyZane,FredStoller",RobertAdetuyi,"스릴러,코미디",6.0,70%,2022.0,https://images.justwatch.com/poster/142028581/...,1시간35분,Tormented by the memories of her late-husband'...,Trouble Sleeping
5649,"SeijiFukushi,HarukaImou,TomoyaMaeno,Hannya,Wat...",YugoSakamoto,"액션,드라마",7.1,76%,2022.0,https://images.justwatch.com/poster/248812373/...,1시간26분,후카미 아키라는 어린 시절 아버지 미쓰오가 누군가에 의해 살해된 이후 일본 지하 세...,야쿠자 어쌔신


In [50]:
watcha1.isnull().sum()

actors          387
director        259
genre           158
imdb_rating     416
just_rating     416
opening_date     35
posterLink       78
runtime          35
synopsis         49
title_kor        35
dtype: int64

In [51]:
# 시놉시스 없는 영화 버기
watcha1 = watcha1.dropna(subset=['synopsis'])

In [52]:
watcha1.isnull().sum()

actors          341
director        214
genre           115
imdb_rating     373
just_rating     373
opening_date      0
posterLink       35
runtime           0
synopsis          0
title_kor         0
dtype: int64

In [53]:
# 결측치 제거 후 인덱스 리셋 
watcha1.reset_index(drop=True, inplace=True)

In [54]:
watcha1['netflix'] = 0
watcha1['disney-plus'] = 0
watcha1['wavve'] = 0
watcha1['watcha'] = 1

In [55]:
watcha1.tail(3)

,actors,director,genre,imdb_rating,just_rating,opening_date,posterLink,runtime,synopsis,title_kor,netflix,disney-plus,wavve,watcha
5598,"HidekiNagai,ItsukiŌtaki,ToshiakiInomata,TokiKo...",TakeshiKushida,"스릴러,공포,로맨스,코미디",6.3,73%,2022.0,https://images.justwatch.com/poster/226608899/...,1시간29분,"시간이 멈춘듯한 아버지가 남긴 사진관에서, 사진 보정 작업을 하고 있는 여성 공포증...",사진사의 여자,0,0,0,1
5599,"VanessaAngel,BillyZane,FredStoller",RobertAdetuyi,"스릴러,코미디",6.0,70%,2022.0,https://images.justwatch.com/poster/142028581/...,1시간35분,Tormented by the memories of her late-husband'...,Trouble Sleeping,0,0,0,1
5600,"SeijiFukushi,HarukaImou,TomoyaMaeno,Hannya,Wat...",YugoSakamoto,"액션,드라마",7.1,76%,2022.0,https://images.justwatch.com/poster/248812373/...,1시간26분,후카미 아키라는 어린 시절 아버지 미쓰오가 누군가에 의해 살해된 이후 일본 지하 세...,야쿠자 어쌔신,0,0,0,1


In [56]:
watcha1.to_csv("watcha.csv")
watcha1.to_excel("watcha.xlsx")

In [71]:
result = pd.concat([wavve2,watcha1], ignore_index=True)
result.tail(3)

,title_kor,opening_date,just_rating,imdb_rating,runtime,synopsis,director,actors,genre,posterLink,netflix,disney-plus,wavve,watcha
16266,사진사의 여자,2022.0,73%,6.3,1시간29분,"시간이 멈춘듯한 아버지가 남긴 사진관에서, 사진 보정 작업을 하고 있는 여성 공포증...",TakeshiKushida,"HidekiNagai,ItsukiŌtaki,ToshiakiInomata,TokiKo...","스릴러,공포,로맨스,코미디",https://images.justwatch.com/poster/226608899/...,0,0,0,1
16267,Trouble Sleeping,2022.0,70%,6.0,1시간35분,Tormented by the memories of her late-husband'...,RobertAdetuyi,"VanessaAngel,BillyZane,FredStoller","스릴러,코미디",https://images.justwatch.com/poster/142028581/...,0,0,0,1
16268,야쿠자 어쌔신,2022.0,76%,7.1,1시간26분,후카미 아키라는 어린 시절 아버지 미쓰오가 누군가에 의해 살해된 이후 일본 지하 세...,YugoSakamoto,"SeijiFukushi,HarukaImou,TomoyaMaeno,Hannya,Wat...","액션,드라마",https://images.justwatch.com/poster/248812373/...,0,0,0,1


In [72]:
result.to_excel("wavve_watcha.xlsx")